## Deep Q Learning (DQN) agent on the CartPole-v0 task from the OpenAI Gym

- 目录
    - 任务目标
    - 算法依赖的库
    - Replay Memory
    - DQN algorithm

### 任务目标

- Agent 必须在两种动作中做出选择（向左或向右移动手推车）这样连接在手推车上的杆子才能保持直立。

### Import Packages

- 首先我们需要 gym 环境:
    `pip install gym`

- 和其他关于 pytorch 的库:
    - neural networks (torch.nn)
    - optimization (torch.optim)
    - automatic differentiation (torch.autograd)
    - utilities for vision tasks (torchvision - a separate package).

In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")

### Replay Memory

- `Transition` 表示环境中单个转换的命名元组。
它实际上将 `(state, action)` 对映射到它们的 `(next_state, reward)` 结果 (状态是屏幕差异图像).

- `ReplayMemory` 一种大小有界的循环缓冲区，用于保存最近观察到的 `Transitions`。
它还实现了一个 `.sample()` 方法，用于选择用于训练的随机一批 `Transitions`。

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### DQN algorithm

我们的目标将是训练一种试图最大化累积回报(reward)的政策(policy) :

$$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$$

where $R_{t_0}$ 是返回值, 折现率 $\gamma$ 应该是一个介于0和1之间的常量，以确保和收敛.
对我们的Agent来说，不确定的遥远未来的回报比我们相当有信心的不久的将来的回报更重要.

Q-learning 背后的主要思想是, 如果我们有一个函数 $Q^*: State \times Action \rightarrow \mathbb{R}$ ,
能告诉我们得到的是什么, 如果我们在一个给定的状态(state)下, 要执行一个行为(action), 
然后我们可以很容易地构造一个策略(policy), 以最大化回报(reward) :

$$\pi^*(s) = \arg\!\max_a \ Q^*(s, a)$$

然而, 我们无法知晓真实世界下的所有情况, 所以我们不能直接求的$Q^*$,
但是, 由于神经网络可以看作通用函数逼近器, 可以利用神经网络训练并近似$Q^*$ :

$$\mathcal{L} = \frac{1}{|B|}\sum_{(s, a, s', r) \ \in \ B} \mathcal{L}(\delta)$$

训练更新规则是 , 对于某些策略(policy), 每个 $Q$ 函数都遵守 Bellman 方程 :

$$Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))$$

等式两边的差称为 时序差分误差(temporal difference error), $\delta$ :

$$\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))$$

为了最小化这个误差, 我们使用 `Huber loss`. 

> 当误差很小的时候, `Huber loss`就像均方误差(MSE), 但当误差很大的时候, 就像绝对平均误差(MAE), 当 $Q$ 的近似有很大的噪声时，这使得它对离群值更加鲁棒.

我们通过从Replay Memory中采样的一批Transition来计算:
 
$$\mathcal{L} = \frac{1}{|B|}\sum_{(s, a, s', r) \ \in \ B} \mathcal{L}(\delta)$$

$$
\begin{split}\text{where} \quad \mathcal{L}(\delta) = \begin{cases}
  \frac{1}{2}{\delta^2}  & \text{for } |\delta| \le 1, \\
  |\delta| - \frac{1}{2} & \text{otherwise.}
\end{cases}\end{split}
$$

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            """conv2d layer output size"""
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

### Input extraction